# ReAct Zero-Shot from PDF Source with LangChain

This notebook shows how to turn a PDF into embeddings, store those embeddings in a persistent local database, query the database directly for relevant pieces of text, and then how to use the database as a source for an LLM hooked into LangChain to create a zero-shot ReAct agent you can ask questions to.

The PDF used in the example is the World Pool-Billiard Association's rule book. You can find the source [here](https://wpapool.com/wp-content/uploads/2017/03/WPA_New_Rules_01MAR2016-fixed-spelling.pdf).

I removed the table of contents pages from the PDF because it was obfuscating the vector search and messing with the ReAct agent results.

## ChromaDB Persistence

Each notebook that uses ChromaDB follows the same pattern for persistence.

If the directory already exists that ChromaDB would be writing it's data to, it will load the existing database. If the directory does not exist, it will create a new database.

If you change parameters that affect the embeddings generation (like swapping in a new PDF file), you'll need to delete the database directory to force a new database to be created.

This can be done by running the following from the root of the repository. If the ChromaDB directory is `data/chromadb/pdf_source`, you'd run the following to delete it:

```sh
rm -rf data/chromadb/pdf_source
```

or if you run into permissions issues:

```sh
sudo rm -rf data/chromadb/pdf_source
```

In [1]:
import os

# ****************** [START] Google Cloud project settings ****************** #
project =  os.getenv('GCP_PROJECT')
location = os.environ.get('GCP_REGION', 'us-central1')
# ******************* [END] Google Cloud project settings ******************* #


# *********************** [START] Embeddings config ************************* #
# set rate limiting options for Vertex AI embeddings
embeddings_requests_per_minute = 100
embeddings_num_instances_per_batch = 5
# *********************** [END] Embeddings config *************************** #


# ********************** [START] data directory config ********************** #
from helpers.files import get_data_dir
data_dir = get_data_dir()

chroma_db_dir = f'{data_dir}/chromadb'
chroma_db_pdf_source_dir = f'{chroma_db_dir}/pdf_source'

pdf_dir = os.path.join(data_dir, 'pdfs')
# *********************** [END] data directory config *********************** #


# ********************** [START] LLM data config **************************** #
from helpers.files import file_exists

collection_name = 'pdf-source'
load_documents = True
if file_exists(chroma_db_pdf_source_dir):
    load_documents = False

chunk_size = 500
chunk_overlap = 150
# *********************** [END] LLM data config ***************************** #


# ******************* [START] vector db retriever config ******************** #
# experiment with:
# - mmr
# - similarity
db_search_type = "similarity"
db_search_kwargs = {"k": 5}
# ********************* [END] vector db retriever config ******************** #


# *********************** [START] PDF tool config *************************** #
# the PDF file to create embeddings for and load into ChromaDB
pdf_file_path = os.path.join(pdf_dir, 'Pool_Billiards_Rules_of_Play_01MAR2016.pdf')

# tool config for the langchain react agent
tool_name = "Read the World Pool-Billiard Association Rules of Play"
tool_description = "use this to read the rules of pool billiards"
# *********************** [END] PDF tool config ***************************** #


# *********************** [START] LLM parameter config ********************** #
# Vertex AI model to use for the LLM
model_name='text-bison@002'

# maximum number of model responses generated per prompt
candidate_count = 1

# determines the maximum amount of text output from one prompt.
# a token is approximately four characters.
max_output_tokens = 2048

# temperature controls the degree of randomness in token selection.
# lower temperatures are good for prompts that expect a true or
# correct response, while higher temperatures can lead to more
# diverse or unexpected results. With a temperature of 0 the highest
# probability token is always selected. for most use cases, try
# starting with a temperature of 0.2.
temperature = 0.1

# top-p changes how the model selects tokens for output. Tokens are
# selected from most probable to least until the sum of their
# probabilities equals the top-p value. For example, if tokens A, B, and C
# have a probability of .3, .2, and .1 and the top-p value is .5, then the
# model will select either A or B as the next token (using temperature).
# the default top-p value is .8.
top_p = 0.8

# top-k changes how the model selects tokens for output.
# a top-k of 1 means the selected token is the most probable among
# all tokens in the model’s vocabulary (also called greedy decoding),
# while a top-k of 3 means that the next token is selected from among
# the 3 most probable tokens (using temperature).
top_k = 40

# how verbose the llm and langchain agent is when thinking
# through a prompt. you're going to want this set to True
# for development so you can debug its thought process
verbose = True
# *********************** [END] LLM parameter config ************************ #


# ********************** [START] Configuration Checks *********************** #
if not project:
    raise Exception('GCP_PROJECT environment variable not set')
# *********************** [END] Configuration Checks ************************ #

## Import and Initialize Vertex AI Client

This will complain about not having cuda drivers and the GPU not being used. You can safely ignore that. If you want to use the GPU, that's possible in Linux with Docker, but you'll need to set up a non-containerized development environment to use GPUs with MacOS.

In [2]:
from google.cloud import aiplatform
import vertexai

vertexai.init(project=project, location=location)

print(f"Vertex AI SDK version: {aiplatform.__version__}")


2023-12-18 00:11:55.697689: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-18 00:11:55.699449: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-18 00:11:55.718301: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 00:11:55.718337: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 00:11:55.718354: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

Vertex AI SDK version: 1.38.1


## Import LangChain

This doesn't actually initialize anything, it just lets us print the version.

In [3]:
import langchain

print(f"LangChain version: {langchain.__version__}")


LangChain version: 0.0.350


## Configure LLM with Vertex AI

In [4]:
from langchain.llms import VertexAI

llm = VertexAI(
    model_name=model_name,
    max_output_tokens=max_output_tokens,
    temperature=temperature,
    top_p=top_p,
    top_k=top_k,
    verbose=verbose,
)


## Initialize Embeddings Function with Vertex AI

There are other options for creating embeddings. I was interested in sticking with Google products here.

In [5]:
from langchain.embeddings import VertexAIEmbeddings

# https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.vertexai.VertexAIEmbeddings.html
embeddings = VertexAIEmbeddings(
    requests_per_minute=embeddings_requests_per_minute,
    num_instances_per_batch=embeddings_num_instances_per_batch,
    model_name = "textembedding-gecko@latest"
)

## Load PDF Documents

In [6]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(pdf_file_path)
documents = loader.load()

## Chunk the PDFs into smaller pieces

In [7]:
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents, chunk_size=1000, chunk_overlap=100):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs


transformed_docs = split_docs(
  documents,
  chunk_size=chunk_size,
  chunk_overlap=chunk_overlap,
)

print(f'Document count: {len(documents)}')
print(f'Transformed document count: {len(transformed_docs)}')

Document count: 25
Transformed document count: 160


## Create Embeddings Database

This is written with persistence and will not re-create the database if it already exists.

In [8]:
from langchain.vectorstores import Chroma

if load_documents:
  # https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.chroma.Chroma.html#langchain.vectorstores.chroma.Chroma.from_documents
  db = Chroma.from_documents(
    transformed_docs,
    embeddings,
    collection_name=collection_name,
    persist_directory=chroma_db_pdf_source_dir,
  )
else:
  db = Chroma(
    persist_directory=chroma_db_pdf_source_dir,
    embedding_function=embeddings,
    collection_name=collection_name,
  )


## Create Retrieval QA Chain

In [9]:
retriever = db.as_retriever(
  search_type=db_search_type,
  search_kwargs=db_search_kwargs,
)

retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

In [10]:
query = "Who shoots first in pool?"
result = retrieval_qa({'query': query})
print(result)

query = f"What does the following mean in simple terms? {result['result']}"
result = retrieval_qa({'query': query})
print(result)


{'query': 'Who shoots first in pool?', 'result': ' The players lag to determine who shoots first.'}
{'query': 'What does the following mean in simple terms?  The players lag to determine who shoots first.', 'result': ' The players shoot at the same time to see who can get their ball closer to the head cushion. Whoever gets their ball closer gets to shoot first.'}


## Build chain

This will create a chat bot interface to talk about the PDF.

In [11]:
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
# https://python.langchain.com/docs/integrations/chat/google_vertex_ai_palm


template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


# https://python.langchain.com/docs/modules/data_connection/retrievers/
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [12]:
import gradio as gr

def chat_response(message, history):
  return chain.invoke(message)

demo = gr.ChatInterface(chat_response)

demo.launch(
  server_name="0.0.0.0",
  server_port=5000,
)

Running on local URL:  http://0.0.0.0:5000

To create a public link, set `share=True` in `launch()`.
